# A guide to Automatic Differentiation
Author: Reinhardt Palko

Date: 19 August 2022

## Introduction

Derivatives, usually in the form of gradients and Hessians, are ubiquitous in the field of machine learning \[1\]. However, working with vectors and tensors, which is usually the case in machine learning, then computing derivatives is not a trivial task. In this notebook, we hope to provide the reader with comprehensive understanding of how one can compute derivatives using automatic differentiation (AD). Automatic differentiation, also called algorithmc differentiation or 'autodiff' for short, is a programming method for computing derivatives of numeric functions. It is important to note that there are various methods for calculating derivatives on a computer, but we try to make it clear as to why one would prefer AD over the other techniques.

### Notation

Some notation to understand the precedings of the notebook.
* $x$ - a scalar, denoted by $x \in \mathbb{R}$,
* $\mathbf{x}$ - a vector, denoted $\mathbf{x} \in \mathbb{R}^n$ where $\mathbf{x} = [x_1 ... x_n]$,
* $\mathsf{X}$ - a tensor,
* $\mathbf{A}$ - a matrix with $m$ rows and $n$ columns, i.e. $\mathbf{A} \in \mathbb{R}^{m \times n}$,
* $A_{i,j}$ - element in row $i$ and column $j$ in matrix $\mathbf{A}$,
* $\mathbf{\theta}$ - model parameters,
* $L(\theta)$ - loss function of a model,
* $\frac{d f}{d x}$ - derivative of $f$ with respect to $x$,
* $\frac{\partial f}{\partial x}$ - partial derivative of $f$ with respect to $x$,
* $\nabla_{\mathbf{x}}f$ or $J$ - Jacobian matrix $J \in \mathbb{R}^{m \times n}$ of $f: \mathbb{R}^n \rightarrow \mathbb{R}^m$
* $\nabla_{\mathbf{x}}^2f$ - Hessian matrix of $f$

### Chain Rule of Calculus

AD relies heavily on the chain rule for calculating derivatives recursively. Here we revise some basics that are necessary for the understanding AD. 
Let $x \in \mathbb{R}$, and then define functions $f: \mathbb{R} \rightarrow \mathbb{R}$ and $g: \mathbb{R} \rightarrow \mathbb{R}$. Suppose that $y = g(x)$ and $z = f(g(x)) = f(y)$, the chain rule then states that 
$$
\begin{equation}
\frac{dz}{dx} = \frac{dz}{dy}\frac{dy}{dx}.
\end{equation}
$$
This can be generalised beyond the scalar case. Suppose now that $\mathbf{x} \in \mathbb{R}^m$, $\mathbf{y} \in \mathbb{R}^n$, $g: \mathbb{R}^m \rightarrow \mathbb{R}^n$, and $f: \mathbb{R}^n \rightarrow \mathbb{R}$. If $\mathbf{y} = g(\mathbf{x})$ and $z = f(\mathbf{y})$, then the chain rule is now given by
$$
\begin{equation}
\frac{\partial z}{\partial x_i} = \sum_{j}\frac{\partial z}{\partial y_j}\frac{\partial y_j}{\partial  x_i}.
\end{equation}
$$
Equivalently, we can rewrite this in vector notation by
$$
\begin{equation}
\frac{\partial z}{\partial x_i} = (\frac{\partial \mathbf{y}}{\partial \mathbf{x}})^T \nabla_{\mathbf{y}}z.
\end{equation}
$$
The above equation is more formally known as a Jacobian-vector product. The backpropagation algorithm we use for neural networks performs such a Jacobian-vector product for each operation in the computational graph (discussed in more detail later on).

## Why do we care about Automatic Differentiation?

For the purpose of computing derivatives of numeric functions on a computer program, we can (1) manually calculate derivatives by hand and code them, (2) use numerical differentiation which uses a finite differences approach, (3) symbolic differentiation which uses expression manipulation and of course (4) automatic differentiation. We argue that for the purpose of obtaining numeric values of derivatives that AD is the best option. We will breifly discuss all these methods and give examples to how one would implement them in Python code. Let $f(x_1, x_2) = \log{x_1}+x_1\cdot x_2-\sin{x_2}$, suppose we want to evaluate the partial derivative of $f$ w.r.t. $x_1$, i.e. $\frac{\partial f}{\partial x_1}$, evaluated at the point $(x_1, x_2) = (2,5)$.

### Manually evaluating derivatives by hand

This procedure is as straightforward as it sounds. Using the differentiation rules we can obtain the symbolic expression 
$$
\begin{equation}
\frac{\partial f}{\partial x_1} = \frac{1}{x_1} + x_2
\end{equation}
$$
Now, evaluating at the point  $(x_1, x_2) = (2,5)$ we obtain:
$$
\begin{equation}
\frac{\partial f}{\partial x_1}|_{x_1=2,x_2=5} = \frac{1}{2} + 5 = 5.5
\end{equation}
$$
Now all we have to do is assign an approriate variable to value of $5.5$ in the code. This approach is conceptually the most simple but requires the manual calculation of each derivative specified in our task. It is clear that this approach can become quite tedious when a function consists of various inputs and outputs. Hence, we need a way of automating this process.

### Numerical Differentiation

Numerical differentiation is the finite difference approach of approximating derivatives using the original function evaluated at some sample points \[1\]. The approach relies on the limit definition of a derivative. Let $f$ be a multivariate function $f: \mathbb{R}^n \rightarrow \mathbb{R}$, we can the approximate the partial derivatives $\nabla_{\mathbf{x}} f = (\frac{\partial f}{\partial x_1}, ..., \frac{\partial f}{\partial x_n})$ using
$$
\begin{equation}
\frac{\partial f(\mathbf{x})}{\partial x_i} \approx \frac{f(\mathbf{x} + h\mathbf{e}_i)-f(\mathbf{x})}{h},
\end{equation}
$$
where $\mathbf{e}_i$ is the $i$-th unit vector and $h > 0$ is a small step size. Going back to our previous example, we can approximate the derivative of $f(x_1, x_2) = \log{x_1}+x_1\cdot x_2-\sin{x_2}$ by setting $\mathbf{x} = 
\begin{bmatrix}
2 & 5
\end{bmatrix}
$ and $\mathbf{e}_1 = 
\begin{bmatrix}
1 & 0
\end{bmatrix}$ with step size $h=10^{-4}$. Using finite differences we obtain the approximation $\frac{\partial f}{\partial x_1}|_{x_1=2,x_2=5} \approx 5.499987500421355$. However, we are concerned with the $\textbf{accurate}$ evaluation of derivatives (at least up to machine precision). This means we require a method that calculates exact derivatives and not approximations thereof. Also, by using finite differences we are committing two sins of numerical analysis;
* "$\textit{thou shalt not add small numbers to big numbers}$" and,
* "$\textit{thou shalt not subtract numbers which are approximately equal}$".


In [ ]:
import numpy as np

# define the function f
def f(x_1, x_2):
    return np.log(x_1) + x_1*x_2 - np.sin(x_2)

# set x_1, x_2 = (2, 5)
x_1, x_2 = (2, 5)
x = np.array([x_1, x_2])

# let e be the i-th unit vector
e = np.zeros(2)
e[0] = 1

# take an appropriate step size h
h = 1e-4

der_x1 = (f(x[0]+h*e[0], x[1]+h*e[1]) - f(x[0], x[1]))/h

print(f'True derivative: 5.5')
print(f'Approximated derivative: {der_x1}')

True derivative: 5.5
Approximated derivative: 5.499987500421355


### Symbolic differentiation

Symbolic differentiation is the process of automatically manipulating expressions of obtaining their respective derivative expressions. This process is carried out by using the rules of differentiation and applying appropriate transformations. For example, we have the common transformations of 
$$
\begin{equation}
\frac{d}{dx}(f(x) + g(x)) = \frac{d}{dx}f(x) + \frac{d}{dx}g(x),
\end{equation}
$$
$$
\begin{equation}
\frac{d}{dx}(f(x) \cdot g(x)) = (\frac{d}{dx}f(x))g(x) + f(x)(\frac{d}{dx}g(x)).
\end{equation}
$$
If we are able to represent these expressions as data structures, then symbolic differentiation is mechanistic process. For our example, if we define the function $f(x_1, x_2) = \log{x_1}+x_1\cdot x_2-\sin{x_2}$ then a computer program would then be able to return the symbolic derivative $\frac{\partial f}{\partial x_1} = \frac{1}{x_1} + x_2$. In fact, this is what happens with computer algebra frameworks such as Mathematica, Maxima and Maple \[1\]. It is then easy to compute the numerical values of these symbolic expressions.
However, obtaining these symbolic derivatives are quite computationally expensive since these symbolic derivatives can become exponentially larger than their original function. This is especially the case when we have multiple compositions of functions, like $f(g(h(f(g(h(x))))))$, computing the derivative with respect to $x$ is not so trivial as before. This problem that sumbolic differentiation suffers from is known as expression swell.

In [ ]:
import numpy as np

# we will give the computer framework the following function as an input
def f(x_1, x_2):
    return np.log(x_1) + x_1*x_2 - np.sin(x_2)

# then the program will apply the appropriate transformations to obtain the symbolic derivative as a function
def df_dx1(x_1, x_2):
    return 1/x_1 + x_2

# set x_1, x_2 = (2, 5)
x_1, x_2 = (2, 5)

der_x1 = df_dx1(2,5)

print(f'True derivative: 5.5')
print(f'Approximated derivative: {der_x1}')

True derivative: 5.5
Approximated derivative: 5.5


In a field such as optimisation, knowing the symbolic expression can become quite usefull of the purpose of analysis. For the purpose of machine learning, we are only interested in an accurate value of the derivative and not the expression itself. This is why require a method that is computationally efficient and accurate. Now we introduce automatic differentiation.

### On Automatic Differentiation

 AD relies on the ability to decompose a program/function into a set of elememtary operations (primitives) for which the derivatives are known to apply the chain rule to this function \[3\]. Given a target function $f: \mathbb{R}^n \rightarrow \mathbb{R}^m$, the $m \times n$ Jacobian matrix $\nabla_\mathbf{x} f$ has $(i, j)^{th}$ component:
 $$
 \begin{equation}
 J_{ij} = \frac{\partial f_i}{\partial x_j}
 \end{equation}
 $$
 The Jacobian matrix defined above contains the partial derivatives of all the outputs, $f_i$, with respect to all the inputs $x_j$. When $f: \mathbb{R}^n \rightarrow \mathbb{R}$, as is the case when have a loss function $J(\theta)$, then the Jacobian matrix is simply the gradient. The partial derivatives that appear in the columns of the Jacobian matrix are also referred to as sensitivities in the literature. For example, $\frac{\partial f}{\partial x_j}$ tells us how sensitive the function $f$ is when we change $x_j$.
In automatic differentiation we want to decompose the function $f$ into elementary operations. Suppose $f$ is a composition of functions, $f=h(g(\mathbf{x}))$ with $\mathbf{x} \in \mathbb{R}^n$, $g: \mathbb{R}^n \rightarrow \mathbb{R}^k$ and $h: \mathbb{R}^k \rightarrow \mathbb{R}^m$. Applying the chain rule we can now rewrite the Jacobian as
$$
\begin{equation}
J_f = J_{h \circ g} = J_h(g(\mathbf{x})) \cdot J_g(\mathbf{x})
\end{equation}
$$
with $(i,j)^{th}$ element:
$$
\begin{equation}
 J_{ij} = \frac{\partial f_i}{\partial x_j} = \frac{\partial h_i}{\partial g_1}\frac{\partial g_1}{\partial x_j} + \frac{\partial h_i}{\partial g_2}\frac{\partial g_2}{\partial x_j} + ... + \frac{\partial h_i}{\partial g_k}\frac{\partial g_k}{\partial x_j}
\end{equation}
$$
If $f$ consists of $L$ such compositions, then we have:
$$
f= f^L \circ f^{L-1} \circ ... \circ f^1.
$$
Similarly, we can write the Jacobian matrix of $f$ as:
$$
J = J_L \circ J_{L-1} \circ ... \circ J_1.
$$
AD consists of two main modes, namely the forward-mode and the reverse-mode. The forward-mode can be seen as computing a recursive series of Jacobian-vector products, whereas the reverse-mode can be seen as a recursive series of vector-Jacobian products. This will be explained in greater detail in the following sections. Before we look at the details of these two modes, it is necessary to obtain a sufficient understanding of computational graphs.

#### Computational Graphs
Computational graphs come in various shapes and forms. Before looking at the details, we find it beneficial to work through an example. Lets look at our original function $f(x_1, x_2) = \log{x_1}+x_1\cdot x_2-\sin{x_2}$ and see how we can decompose it to create a computational graph. The function is essentially applying operations to our inputs $x_1$ and $x_2$ in some sequential manner. An operation is defined as a simple fucntion of one or more variables \[4\]. One operation applied on $x_1$ is that of the $\log$-operator before being multiplied with $x_2$ and being added together. This means that $x_1$ undergoes 3 operations, where the order is very specific since we have to multiply $x_1$ and $x_2$ before we can add it to $\log{x_1}$, otherwise we change the expression of $f$. We can this write sequence of operations more formally as:
$$
\begin{align}
(x_1, x_2)  &\rightarrow (\log{x_1},x_2) \\
 &\rightarrow (\log{x_1},x_1 \cdot x_2, \sin{x_2}) \\
 &\rightarrow (\log{x_1} + x_1 \cdot x_2, \sin{x_2}) \\
 &\rightarrow (\log{x_1} + x_1 \cdot x_2 - \sin{x_2}) \\
\end{align}
$$
This sequence of operations that we applied is known as an evaluation trace, which is what is used in every AD framework in some way. Another way to view this trace is by constructing a graph. Using the methodology defined above, we construct a computational graph for $f$. 
![Demonstrational%20Computation%20Graph.png](attachment:Demonstrational%20Computation%20Graph.png)

Now that we have an understanding of the underlying process of the computational graph, we can now define a fomnal method of constructing such a graph that is unambiguous. We adpot the three-notation from \[5\] to construct a function $f: \mathbb{R}^n \rightarrow \mathbb{R}^m$ using variables $v_i$ such that 
* $v_{i-n}=x_i$, $i=1,...,n$ are the input variables,
* $v_{i}$, $i=1,...,l$ are the intermediate variables,
* $v_{l-i} = y_{m-i}$, $i=m-1,...,0$ are the output variables.
Using this notation defined above, we can now construct the primal trace in the table below. A primal trace refers to the elementary operations on our variables, whereas the tangent trace will refer to the derivative version of these operations. We can think of $v_{i}$ as the numeric values obtained from the elementary operations.


| Variable | Operation      | Numeric Value   |
|:--------:|:--------------:|:---------------:|
| $v_{-1}$ | $x_1$          | $2$             |
| $v_{0}$  | $x_2$          | $5$             |
| $v_{1}$  | $\log{v_{-1}}$ |  $\log{2}$      |
| $v_{2}$  | $v_{-1}\times v_0$ | $2\times 5$ |
| $v_{3}$  | $\sin{v_0}$    | $\sin{5}$       |
| $v_{4}$  | $v_1 + v_2$    | $0.69 + 10$    |
| $v_{5}$  | $v_4 - v_3$    | $10.69+0.95$  |

Note that all variables $v_{i}$ where $i \leq 0$ are the input variables which are initialised to some input values. The intermediate variables $v_{i}$ for $i=1,...,n-l$ then define the operations that are used to form our function $f$. The remaining variables now define the output variables, in the case where $f: \mathbb{R}^n \rightarrow \mathbb{R}$ we have a scalar output. Meaning that $v_{n}$ will determine the output value of $f$. The computationial graph for the primal trace is given below.

![Computational%20Graph%20%28Primals%20only%29.png](attachment:Computational%20Graph%20%28Primals%20only%29.png)

Note that in the above computational graph we write the expression above the arrow for each variable $v_{i} \forall i$.Also, we have nodes which have multiple inputs as is the case for $v_{2}$ which has $v_{-1}$ and $v_{0}$ as inputs. Any such occurence when a node contains multiple inputs is called a fork. Here we show that when a fork occurs that there are multiple inputs. Although this may redundant it will make it easier to understand when we obtain values for the derivatives.

#### Forward-mode Automatic Differentiation

The forward accumulation mode of AD is discussed first since it is the most conceptually simple of the two. To compute the derivative of $f$ with respect to a variable $x_j$ we express the tangent value of $v_i$ by
$$
\begin{equation}
\dot{v}_i = \frac{\partial v_i}{\partial x_j}
\end{equation}.
$$
By applying the chain rule to every elemtary operation in defined the forward primal trace, we obtain the corresponding forward tangent trace. Suppose that we want to compute the derivative of $\frac{\partial f}{\partial x_1}$ at the point $(x_1=2,x_2=5)$. In the same way we initialised the primals, we need to initialise the tangents. We do this by setting $\dot{x}_1 = 1$ and $\dot{x}_2 = 0$, since we want to compute the derivative with respect to $x_1$. The table below demonstrates how we obtain the forward primal trace along with the forward tangent trace.

| Forward Primal Trace   | Primal Value              | Forward Tangent Trace          | Tangent Value          |
|:----------------------:|:--------------------------|:-------------------------------|:-----------------------|
|$v_{-1} = x_1$          |$= 2$                      | $\dot{v}_{-1} = \dot{x}_1$     |$ = 1$                  |
|$v_{0} = x_2 $          |$= 5$                      | $\dot{v}_{0} = \dot{x}_2$      |$= 5$                   |
|$v_1 = \log{v_{-1}}$    |$=\log{2}$                 | $\dot{v}_{1} = \frac{\dot{v}_{-1}}{v_{-1}}$|$= \frac{1}{2}$   |
|$v_{2}=v_{-1}\times v_0$|$= 2\times 5$              | $\dot{v}_2 = \dot{v}_{-1}\times v_0 + \dot{v}_{0}\times v_{-1}$|$=1\times 5+0\times 2$|
|$v_{3}=\sin{v_0}$       |$=\sin{5}$                 | $\dot{v}_3 = \dot{v}_0\cos{v_{0}}$|$=0\times \cos{5}$    |
|$v_{4}=v_1 + v_2$       |$=0.693 + 10$              | $\dot{v}_4=\dot{v}_1 + \dot{v}_2$|$=0.5+5$               |
|$v_{5}=v_4 - v_3$       |$10.693+0.959$             | $\dot{v}_5=\dot{v}_4 - \dot{v}_3$|$=5.5-0$          |

It is imperative to understand the notation used in the computational graph below. The primal traces (black) are displayed above the edges, while the tangent traces (blue) are displayed below. Also, at each instance where a node receives multiple inputs, we encounter a fork. In the case of a fork we need to compute the tangent values by summing the partial derivative with respect to each input. For example, the node $v_2$ has two inputs, $v_1$ and $v_2$. This means that we have to compute the partial derivative with respect to each of these nodes. We denote these two derivative values with a superscript ${*}$, i.e. $\dot{v}^{*}_2 = \dot{v}_{-1}\times v_0$ and $\dot{v}^{*}_2 = \dot{v}_{0}\times v_{-1}$. Summing over these two $\dot{v}^{*}_2$ values give us the full tangent value $\dot{v}_2 = \dot{v}_{-1}\times v_0 + \dot{v}_{0}\times v_{-1}$.

![Computational%20Graph%20-%20Forward%20mode.drawio.png](attachment:Computational%20Graph%20-%20Forward%20mode.drawio.png)

#### Where does the Chain Rule come in?
In the literature, AD is cited to be an efficient algorithm due to its recursive nature. This recursion is seen due to the chain rule. Note, we defined $\dot{v}_i = \frac{\partial v_i}{\partial x_j}$, which means that each tangent $\dot{v}_i$ in our graph is the derivative with respect to $x_1$. For example, $\dot{v}_4 = \frac{\partial v_4}{\partial x_1}$ by definition. But in our graph we see that $v_4$ has no direct input from the node $x_1$, we only have $\dot{v}_4 = \dot{v}_1 + \dot{v}_1$. In fact, none of the intermediate tangent values $\frac{\partial v_5}{\partial x_1},\frac{\partial v_4}{\partial x_1},\frac{\partial v_3}{\partial x_1},\frac{\partial v_2}{\partial x_1},\frac{\partial v_1}{\partial x_1}$ are cleary defined. Only the input nodes $\frac{\partial v_{-1}}{\partial x_1},\frac{\partial v_0}{\partial x_1}$ are defined by their initialisation $\frac{\partial v_{-1}}{\partial x_1}=1$ and $\frac{\partial v_0}{\partial x_1}=0$. Hence, we cannot take the derivative of $v_4$ w.r.t. $x_1$, we can only take the derivative w.r.t. its input nodes $v_1$ and $v_2$. i.e. $\frac{\partial v_4}{\partial v_1}$ and $\frac{\partial v_4}{\partial v_2}$ respectively. Now,
$$
\begin{align}
\dot{v}_4 &= \dot{v}_1 + \dot{v}_2 \\
          &= \frac{\partial v_1}{\partial x_1} + \frac{\partial v_2}{\partial x_1} \\
          &= \frac{\partial v_1}{\partial v_{-1}}\frac{\partial v_{-1}}{\partial x_1} + \underbrace{\frac{\partial v_2}{\partial v_0}\frac{\partial v_0}{\partial x_1} + \frac{\partial v_2}{\partial v_{-1}}\frac{\partial v_{-1}}{\partial x_1}}_{\frac{\partial v_2}{\partial x_1}} \text{  (by the chain rule)} \\
          &= \frac{\partial v_1}{\partial v_{-1}}\dot{v}_{-1} + \frac{\partial v_2}{\partial v_0}\dot{v}_{0} + \frac{\partial v_2}{\partial v_{-1}}\dot{v}_{-1} \text{  (by definition)}
\end{align}
$$
Here we were able to decompose each $\dot{v}_1$ until we were able to obtain the initialised values of $\dot{v}_{-1} = \frac{\partial v_{-1}}{\partial x_1} = 1$ and $\dot{v}_{0} = \frac{\partial v_0}{\partial x_1} = 0$.  However, we have not yet discussed $\frac{\partial v_1}{\partial v_{-1}}$ and $\frac{\partial v_2}{\partial v_{0}}$. Recall that $v_1 = v_{-1} \times v_{0}$, therefore:
$$
\begin{equation}
\frac{\partial v_2}{\partial v_{-1}} = v_0.
\end{equation}
$$
Similarly,
$$
\begin{equation}
\frac{\partial v_2}{\partial v_{0}} = v_{-1}.
\end{equation}
$$

##### Forward-mode as Jacobian-vector product interpretation 
The function we have used up until this point is of the form $f: \mathbb{R}^2 \rightarrow \mathbb{R}$ which means that we can compute all partial derivatives to obtain the Jacobian $J \in \mathbb{R}^{1 \times 2}$. Hence, for $y =f(x_1, x_2) = \log{x_1}+x_1\cdot x_2-\sin{x_2}$ we have:
$$
\begin{align}
J = 
    \begin{bmatrix}
        \frac{\partial y}{\partial x_1} & \frac{\partial y}{\partial x_2}
    \end{bmatrix}
\end{align}
$$


Note that using our current initialisation scheme, where we set $\frac{\partial y}{\partial x_1} = 1$ and $\frac{\partial y}{\partial x_2} = 0$, we were able to compute the partial derivative of $\frac{\partial y}{\partial x_1}$ which is the first column of our Jacobian matrix $J$. To compute the second column of $J$, we can use the same process of the computational graph with the initialisation scheme $\frac{\partial y}{\partial x_1} = 0$ and $\frac{\partial y}{\partial x_2} = 1$. Hence, can use forward-mode AD to compute the full Jacobian matrix by evaluating each colummn. In general, for a function $f: \mathbb{R}^n \rightarrow \mathbb{R}^m$ we can compute the full Jacobian $J \in \mathbb{R}^{m \times n}$ in $n$ iterations of forward-mode AD ($n$ corresponds to the number of columns in $J$, and the number of inputs in $f$). If we use the notation, $\dot{x}_{1} = \dot{v}_{-1} = \frac{\partial v_{-1}}{\partial x_1}$ and $\dot{x}_{2} = \dot{v}_{0} = \frac{\partial v_{0}}{\partial x_1}$, then we can use vector notation $\dot{\mathbf{x}} = \begin{bmatrix} \dot{x}_1  & \dot{x}_2 \end{bmatrix}$. This means that our initialisation scheme can be written as $\dot{\mathbf{x}} = \mathbf{e}_i$, where $\mathbf{e}_i$ is the $i^{\text{th}}$ unit vector. Therefore, with one iteraton of forward-mode AD, we are evaluating the Jacobian $J$ at the point $\dot{\mathbf{x}} = \mathbf{e}_i$, i.e.
$$
\begin{equation}
    J = 
    \begin{bmatrix}
        \frac{\partial f_1}{\partial x_1} & \dots & \frac{\partial f_1}{\partial x_n} \\
        \vdots & \ddots & \vdots \\
        \frac{\partial f_m}{\partial x_1} & \dots & \frac{\partial f_m}{\partial x_n} \\
    \end{bmatrix}\bigg|_{\dot{\mathbf{x}}=\mathbf{e}_i}
\end{equation}
$$
Furthermore, if we were to compute the Jacobian-vector product of $J$ and an arbitrary vector $\mathbf{u}$
$$
\begin{equation}
J \mathbf{u} = \begin{bmatrix}
        \frac{\partial f_1}{\partial x_1} & \dots & \frac{\partial f_1}{\partial x_n} \\
        \vdots & \ddots & \vdots \\
        \frac{\partial f_m}{\partial x_1} & \dots & \frac{\partial f_m}{\partial x_n} \\
    \end{bmatrix}
    \begin{bmatrix}
        u_1 \\
        \vdots \\
        u_n \\
    \end{bmatrix}
\end{equation}
$$
by simply initialising $\dot{\mathbf{x}} = \mathbf{u}$. Thus, forward-mode AD provides a simple and efficient way to compute Jacobian-vector products in just one forward pass.  
Forward-mode AD is efficient for functions $f: \mathbb{R}^n \rightarrow \mathbb{R}^m$ where $n >> m$

#### Reverse-mode Automatic Differentiation
We now look at the reverse-mode of AD, which calculates the primals first in a forward sweep and then the derivatives in a reverse sweep. In forward-mode, we denoted the derivative values by $\frac{\partial v_i}{\partial x_j}$ and are referred to as tangents. To distinguish between reverse-mode and forward-mode, we denote derivatives by 
$$
\begin{equation}
\bar{v}_i = \frac{\partial y_j}{\partial v_i}
\end{equation}
$$ 
and are referred to as adjoints in the reverse mode. Instead of iterating through the computational graph and computing the primal and derivative in the same step, we will now compute all primals and record this evaluation trace (we refer to this as a tape in the literature). This computation of all primals is reffered to as the forward sweep. Now, to calculate the derivative values - or adjoints - we traverse backwards in this evaluation trace from the output to the input. This backward traversal of the evaluation trace is why we call it the reverse-mode.
The table below demonstrates the forward primal trace.

| Forward Primal Trace   | Primal Value              |
|:-----------------------|:--------------------------|
|$v_{-1} = x_1$          |$= 2$                      |
|$v_{0} = x_2 $          |$= 5$                      |
|$v_1 = \log{v_{-1}}$    |$=\log{2}$                 |
|$v_{2}=v_{-1}\times v_0$|$= 2\times 5$              |
|$v_{3}=\sin{v_0}$       |$=\sin{5}$                 |
|$v_{4}=v_1 + v_2$       |$=0.693 + 10$              |
|$v_{5}=v_4 - v_3$       |$10.693+0.959$             |

For the reverse mode, we now initialise the final output derivative to 1, i.e.
$$
\begin{equation}
\bar{v}_5 = \frac{\partial y}{\partial v_5} = 1.
\end{equation}
$$
If we had multiple outputs, $\mathbf{y} = \begin{bmatrix} y_1 \dots y_m \end{bmatrix}$, then we initialise the adjoint in terms of the output variable $y_j$ to 1 and the rest are set to 0.

The reverse adjoint trace is given in the table below.

| Reverse Adjoint Trace  | Reverse Adjoint Value    |
|:-----------------------|:--------------------------|
|$\bar{v}_5 = \bar{y}$          |$= 1$  (initialisation)           |
|$\bar{v}_4 = \bar{v}_5 \frac{\partial v_5}{\partial v_4} = \bar{v}_5 \times 1$         |$=1\times 1= 1$     |
|$\bar{v}_3 = \bar{v}_5 \frac{\partial v_5}{\partial v_3} = \bar{v}_5 \times (-1)$      |$=1\times (-1)= -1$    |
|$\bar{v}_2 = \bar{v}_4 \frac{\partial v_4}{\partial v_2} = \bar{v}_4\times 1$      |$=1 \times 1= 1$    |
|$\bar{v}_1 = \bar{v}_4 \frac{\partial v_4}{\partial v_`} = \bar{v}_4\times 1$      |$=1 \times 1= 1$    |
|$\bar{v}^{*}_0 = \bar{v}_3 \frac{\partial v_3}{\partial v_0} = \bar{v}_3\cos{v_0}$      |$=-1 \times \cos{5}= -0.284$    |
|$\bar{v}^{*}_{-1} = \bar{v}_2 \frac{\partial v_2}{\partial v_{-1}} = \bar{v}_3\times v_0$      |$=1 \times 1= 1$    |
|$\bar{v}^{*}_0 = \bar{v}_2 \frac{\partial v_2}{\partial v_0} = \bar{v}_2\times v_{-1}$      |$=2 \times 1= 2$    |
|$\bar{v}^{*}_{-1} = \bar{v}_1 \frac{\partial v_1}{\partial v_{-1}} = \frac{\bar{v}_1}{v_{-1}}$      |$=\frac{1}{2}= 0.5$    |
|$\bar{x}_1 = \bar{v}^{*}_{-1}+\bar{v}^{*}_{-1}$|$=5 + 0.5 = 5.5$ |
|$\bar{x}_2 = \bar{v}^{*}_{0}+\bar{v}^{*}_{0}$|$=2 - 0.284 = 1.716$ |

From the table above we see that by using the reverse-mode of AD, we obtain the derivative values of one output with respect to all inputs (where number of inputs is equal to $n$). This means we can now construct the full Jacobian $J$, with $m$ iterations of the reverse-mode. Thus, the distinguishing factor between the forward-mode and reverse-mode is that the forward-mode builds the Jacobian one column at a time, whereas the reverse-mode builds the Jacobian one row at a time. For functions $f: \mathbb{R}^n \rightarrow \mathbb{R}^m$ where $n>>m$ the reverse mode is more computationally efficient. For machine learning, we will always use the reverse mode since we are optmising an loss function $L(\theta)$ with respect to an array of parameters $\theta$. In the computational graph below, we first compute the forward pass of all values (black) and then do a reverse sweep to determine the value of the adjoints (blue).

![Computational%20Graph%20-Reverse%20mode.drawio.png](attachment:Computational%20Graph%20-Reverse%20mode.drawio.png)

## References

* [1] @article{https://doi.org/10.48550/arxiv.1502.05767,
  doi = {10.48550/ARXIV.1502.05767},
  
  url = {https://arxiv.org/abs/1502.05767},
  
  author = {Baydin, Atilim Gunes and Pearlmutter, Barak A. and Radul, Alexey Andreyevich and Siskind, Jeffrey Mark},
  
  keywords = {Symbolic Computation (cs.SC), Machine Learning (cs.LG), Machine Learning (stat.ML), FOS: Computer and information sciences, FOS: Computer and information sciences, G.1.4; I.2.6, 68W30, 65D25, 68T05},
  
  title = {Automatic differentiation in machine learning: a survey},
  
  publisher = {arXiv},
  
  year = {2015},
  
  copyright = {arXiv.org perpetual, non-exclusive license}
}
*  \[2\]	Charles C. Margossian:
A review of automatic differentiation and its efficient implementation. WIREs Data Mining Knowl. Discov. 9(4) (2019)
* \[3\] @article{DBLP:journals/corr/abs-1810-11530,
  author    = {Bart van Merri{\"{e}}nboer and
               Olivier Breuleux and
               Arnaud Bergeron and
               Pascal Lamblin},
  title     = {Automatic differentiation in {ML:} Where we are and where we should
               be going},
  journal   = {CoRR},
  volume    = {abs/1810.11530},
  year      = {2018},
  url       = {http://arxiv.org/abs/1810.11530},
  eprinttype = {arXiv},
  eprint    = {1810.11530},
  timestamp = {Wed, 31 Oct 2018 14:24:29 +0100},
  biburl    = {https://dblp.org/rec/journals/corr/abs-1810-11530.bib},
  bibsource = {dblp computer science bibliography, https://dblp.org}
}
* \[4\] @book{Goodfellow-et-al-2016,
    title={Deep Learning},
    author={Ian Goodfellow and Yoshua Bengio and Aaron Courville},
    publisher={MIT Press},
    note={\url{http://www.deeplearningbook.org}},
    year={2016}
}
* \[5\] @book{griewank2008evaluating,
  title={Evaluating derivatives: principles and techniques of algorithmic differentiation},
  author={Griewank, Andreas and Walther, Andrea},
  year={2008},
  publisher={SIAM}
}